In [1]:
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import ast
from collections import defaultdict
import numpy as np

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Modeling: Transfer learning en ML

In [2]:
models = {
    'SVR': SVR(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'Lasso': Lasso(),
    'LinearRegression': LinearRegression(),
    'KNN': KNeighborsRegressor(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'XGBoost': XGBRegressor()
}

In [3]:
bests_models = pd.read_excel('../data/final_data/bests_models_results.xlsx')
bests_models

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter2-cov,FL,GradientBoosting,{'model__n_estimators': 50},0.799110,1972.905810,2408.717196
1,new_values_log,iter2-cov,GA,Lasso,{'model__alpha': 1.0},0.881413,11931.292662,15234.031189
2,all_log,iter2-cov,NC,LinearRegression,{},0.868628,10247.423473,13541.573294
3,all_pca,iter2-cov,NJ,LinearRegression,{},0.595462,31778.200052,35635.501147
4,new_values,iter2-cov,NY,GradientBoosting,{'model__n_estimators': 100},0.898738,502.854126,604.001534
5,new_values,iter2-cov,SC,Lasso,{'model__alpha': 1.0},0.858107,7297.583055,9056.313126
6,new_values_log,iter2-cov,VA,Lasso,{'model__alpha': 1.0},0.916111,13800.867247,17486.793986


In [4]:
mejores_modelos = bests_models[['State','Modelo','Parameters','Version_data']].to_dict(orient='records')

modelos_dict = {}
for diccionario in mejores_modelos:
    modelos_dict[diccionario.get('State')] = {'Modelo': diccionario.get('Modelo'), 'Parameters': diccionario.get('Parameters'),
    'Version_data': diccionario.get('Version_data')}
modelos_dict

{'FL': {'Modelo': 'GradientBoosting',
  'Parameters': "{'model__n_estimators': 50}",
  'Version_data': 'new_values_log'},
 'GA': {'Modelo': 'Lasso',
  'Parameters': "{'model__alpha': 1.0}",
  'Version_data': 'new_values_log'},
 'NC': {'Modelo': 'LinearRegression',
  'Parameters': '{}',
  'Version_data': 'all_log'},
 'NJ': {'Modelo': 'LinearRegression',
  'Parameters': '{}',
  'Version_data': 'all_pca'},
 'NY': {'Modelo': 'GradientBoosting',
  'Parameters': "{'model__n_estimators': 100}",
  'Version_data': 'new_values'},
 'SC': {'Modelo': 'Lasso',
  'Parameters': "{'model__alpha': 1.0}",
  'Version_data': 'new_values'},
 'VA': {'Modelo': 'Lasso',
  'Parameters': "{'model__alpha': 1.0}",
  'Version_data': 'new_values_log'}}

# Aumented data OverSampling

In [5]:
results_dict = {
    'Version_data': [],
    'Version_iteracion': 'iter3-OverSampling',
    'State': [],
    'Modelo': [],
    'Parameters': [],
    'R2': [],
    'MAE': [],
    'RMSE': []
}
States = bests_models.State.unique()
directory = '../data/final_data/'

for state in States:
    results_dict['Version_data'].append(modelos_dict[state].get('Version_data'))
    results_dict['Modelo'].append(modelos_dict[state].get('Modelo'))
    results_dict['Parameters'].append(modelos_dict[state].get('Parameters'))
    results_dict['State'].append(state)

    filename = f"data_{state}.csv"
    filepath = os.path.join(directory, filename)
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
        X = data.drop(['MedianPrice', 'State'], axis=1)
        y = data['MedianPrice'] 

        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        
        # Sobremuestreo utilizando Random Oversampling
        ros = RandomOverSampler()
        X_train, y_train = ros.fit_resample(X_train, y_train)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        model_name = modelos_dict[state].get('Modelo')
        model = models.get(model_name)

        if model is not None:
            parametros = modelos_dict[state].get('Parameters')

            if isinstance(model, type):
                model.set_params(**parametros)

            # Entrenar el modelo y hacer predicciones
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Calcular scores
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = mean_squared_error(y_test, y_pred, squared=False)

            results_dict['R2'].append(r2)
            results_dict['MAE'].append(mae)
            results_dict['RMSE'].append(rmse)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.274e+10, tolerance: 9.886e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.709e+08, tolerance: 1.050e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.765e+10, tolerance: 1.855e+08

In [6]:
results_df = pd.DataFrame(results_dict)
results_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.876481,18908.855316,24490.409269
1,new_values_log,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.908647,14088.486071,17574.803101
2,all_log,iter3-OverSampling,NC,LinearRegression,{},0.859973,17323.842658,22745.449226
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314
4,new_values,iter3-OverSampling,NY,GradientBoosting,{'model__n_estimators': 100},0.963202,13145.824099,15342.518510
5,new_values,iter3-OverSampling,SC,Lasso,{'model__alpha': 1.0},0.922525,14527.211771,17712.299359
6,new_values_log,iter3-OverSampling,VA,Lasso,{'model__alpha': 1.0},0.948530,18422.090630,24200.519020


In [7]:
models_df =  pd.concat([bests_models, results_df], axis=0).sort_values(by=['State','R2'], ascending = [True,False] )
models_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.876481,18908.855316,24490.409269
0,new_values_log,iter2-cov,FL,GradientBoosting,{'model__n_estimators': 50},0.799110,1972.905810,2408.717196
1,new_values_log,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.908647,14088.486071,17574.803101
1,new_values_log,iter2-cov,GA,Lasso,{'model__alpha': 1.0},0.881413,11931.292662,15234.031189
2,all_log,iter2-cov,NC,LinearRegression,{},0.868628,10247.423473,13541.573294
2,all_log,iter3-OverSampling,NC,LinearRegression,{},0.859973,17323.842658,22745.449226
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314
3,all_pca,iter2-cov,NJ,LinearRegression,{},0.595462,31778.200052,35635.501147
4,new_values,iter3-OverSampling,NY,GradientBoosting,{'model__n_estimators': 100},0.963202,13145.824099,15342.518510
4,new_values,iter2-cov,NY,GradientBoosting,{'model__n_estimators': 100},0.898738,502.854126,604.001534


# Undersampling

In [8]:
results_dict = {
    'Version_data': [],
    'Version_iteracion': 'iter3-OverSampling',
    'State': [],
    'Modelo': [],
    'Parameters': [],
    'R2': [],
    'MAE': [],
    'RMSE': []
}
States = bests_models.State.unique()
directory = '../data/final_data/'

for state in States:
    results_dict['Version_data'].append(modelos_dict[state].get('Version_data'))
    results_dict['Modelo'].append(modelos_dict[state].get('Modelo'))
    results_dict['Parameters'].append(modelos_dict[state].get('Parameters'))
    results_dict['State'].append(state)

    filename = f"data_{state}.csv"
    filepath = os.path.join(directory, filename)
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
        X = data.drop(['MedianPrice', 'State'], axis=1)
        y = data['MedianPrice'] 

        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        
        # Sobremuestreo utilizando Random Oversampling
        ros = RandomUnderSampler()
        X_train, y_train = ros.fit_resample(X_train, y_train)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        model_name = modelos_dict[state].get('Modelo')
        model = models.get(model_name)

        if model is not None:
            parametros = modelos_dict[state].get('Parameters')

            if isinstance(model, type):
                model.set_params(**parametros)

            # Entrenar el modelo y hacer predicciones
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Calcular scores
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = mean_squared_error(y_test, y_pred, squared=False)

            results_dict['R2'].append(r2)
            results_dict['MAE'].append(mae)
            results_dict['RMSE'].append(rmse)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.686e+09, tolerance: 3.295e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.709e+08, tolerance: 1.050e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.913e+09, tolerance: 9.277e+07

In [9]:
results_df = pd.DataFrame(results_dict)
results_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.860585,20066.742565,26018.565332
1,new_values_log,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.910133,13845.975322,17431.324699
2,all_log,iter3-OverSampling,NC,LinearRegression,{},0.851522,17800.740778,23421.780145
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314
4,new_values,iter3-OverSampling,NY,GradientBoosting,{'model__n_estimators': 100},0.944569,15489.633917,18830.387179
5,new_values,iter3-OverSampling,SC,Lasso,{'model__alpha': 1.0},0.922525,14527.211771,17712.299359
6,new_values_log,iter3-OverSampling,VA,Lasso,{'model__alpha': 1.0},0.947649,18596.490733,24406.687688


In [10]:
models_df =  pd.concat([models_df, results_df], axis=0).sort_values(by=['State','R2'], ascending = [True,False])
models_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.876481,18908.855316,24490.409269
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.860585,20066.742565,26018.565332
0,new_values_log,iter2-cov,FL,GradientBoosting,{'model__n_estimators': 50},0.799110,1972.905810,2408.717196
1,new_values_log,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.910133,13845.975322,17431.324699
1,new_values_log,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.908647,14088.486071,17574.803101
1,new_values_log,iter2-cov,GA,Lasso,{'model__alpha': 1.0},0.881413,11931.292662,15234.031189
2,all_log,iter2-cov,NC,LinearRegression,{},0.868628,10247.423473,13541.573294
2,all_log,iter3-OverSampling,NC,LinearRegression,{},0.859973,17323.842658,22745.449226
2,all_log,iter3-OverSampling,NC,LinearRegression,{},0.851522,17800.740778,23421.780145
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314


In [11]:
bests_models_idx = models_df.groupby('State')['R2'].idxmax()
bests_models = models_df.loc[bests_models_idx].groupby('State').first().reset_index()
bests_models

,State,Version_data,Version_iteracion,Modelo,Parameters,R2,MAE,RMSE
0,FL,new_values_log,iter3-OverSampling,GradientBoosting,{'model__n_estimators': 50},0.876481,18908.855316,24490.409269
1,GA,new_values_log,iter3-OverSampling,Lasso,{'model__alpha': 1.0},0.910133,13845.975322,17431.324699
2,NC,all_log,iter2-cov,LinearRegression,{},0.868628,10247.423473,13541.573294
3,NJ,all_pca,iter3-OverSampling,LinearRegression,{},0.831319,39046.745229,46977.315314
4,NY,new_values,iter3-OverSampling,GradientBoosting,{'model__n_estimators': 100},0.963202,13145.824099,15342.518510
5,SC,new_values,iter3-OverSampling,Lasso,{'model__alpha': 1.0},0.922525,14527.211771,17712.299359
6,VA,new_values_log,iter3-OverSampling,Lasso,{'model__alpha': 1.0},0.948530,18422.090630,24200.519020


In [12]:
bests_models.to_excel('../data/final_data/final_bests_models_results.xlsx', index = False)

# Intentando TL 

No se puede, ya que los modelos de ML no son modelos que usan pesos de entrenamiento como las NL. Ademas puede no ser óptimo, ya que los datos de un estado pueden tener características únicas que el modelo haya aprendido a capturar durante el entrenamiento. Por lo tanto, es posible que el rendimiento de las predicciones en un estado diferente no sea tan bueno como en el estado para el cual se entrenó el modelo